In [20]:
!pip install -q altair_saver


You should consider upgrading via the '/usr/local/opt/python/bin/python3.7 -m pip install --upgrade pip' command.


In [21]:
!npm install --silent vega-lite vega-cli canvas

[canvas] Success: "/Users/mash2/Dropbox (PAC LAB)/codes/ionic_projects/SARAV3/SARA_modular/node_modules/canvas/build/Release/canvas.node" is installed via remote
+ vega-lite@4.17.0
+ vega-cli@5.17.0
+ canvas@2.6.1
updated 3 packages and audited 1608 packages in 46.349s
found 982 vulnerabilities (720 low, 5 moderate, 257 high)
  run `npm audit fix` to fix them, or `npm audit` for details


In [22]:
import altair as alt
from vega_datasets import data
cars = data.cars.url

chart = alt.Chart(cars).mark_bar().encode(
  x=alt.X('Miles_per_Gallon:Q', bin=True),
  y='count()',
)

chart.display()

alt.Chart(...)

In [23]:
for fmt in ['json', 'vg.json', 'html', 'png', 'svg', 'pdf']:
  save(chart, f'chart.{fmt}')
!ls -lh chart.*

-rw-r--r--  1 mash2  staff   911B Nov  1 13:33 chart.html
-rw-r--r--  1 mash2  staff   464B Nov  1 13:33 chart.json
-rw-r--r--  1 mash2  staff    22K Nov  1 13:33 chart.pdf
-rw-r--r--  1 mash2  staff    14K Nov  1 13:33 chart.png
-rw-r--r--  1 mash2  staff    10K Nov  1 13:33 chart.svg
-rw-r--r--  1 mash2  staff   4.4K Nov  1 13:33 chart.vg.json


In [24]:
from vega_datasets import data

temps = data.seattle_temps()
temps.head()

,date,temp
0,2010-01-01 00:00:00,39.4
1,2010-01-01 01:00:00,39.2
2,2010-01-01 02:00:00,39.0
3,2010-01-01 03:00:00,38.9
4,2010-01-01 04:00:00,38.8


In [25]:
temps.dtypes

date    datetime64[ns]
temp           float64
dtype: object

In [26]:
temps = temps[temps.date < '2010-01-15']

alt.Chart(temps).mark_line().encode(
    x='date:T',
    y='temp:Q'
)

alt.Chart(...)

In [27]:
def getSqlConfigFromJSON(configFileName):
    """
    Loads full mysql connect object from database returns the value as a JSON object. Format of the JSON object 
    is the following:
    
        {
            "host": "hostname.com",
            "port": 99999,
            "user": "root",
            "passwd": "passworkd",
            "database": "database_or_schema_name"
        }
    """
    
    with open(configFileName) as f:
        mysqlConnectObject = json.load(f)
        
    return mysqlConnectObject



def getScreenUsageDataFrame(user_id, start_unix_time, end_unix_time):
    """
    Select screen usage data points into the test database.
    """

    print('Fetching data')
    
    #get sql connect config
    mysqlConnectObject = getSqlConfigFromJSON('./awareSqlConfig.json')

    # connect to db
    db = mysql.connect(
        host = mysqlConnectObject["host"],
        port = mysqlConnectObject["port"],
        user = mysqlConnectObject["user"],
        passwd = mysqlConnectObject["passwd"],
        database = mysqlConnectObject["database"]
    )
    cursor = db.cursor()

    # fetch data.
    where_clause = "device_id='" + user_id + "'"
    where_clause = where_clause + " AND timestamp > " + str(start_unix_time) 
    where_clause = where_clause + " AND timestamp < " + str(end_unix_time) 
    where_clause = "(" + where_clause +  ")"
    cursor.execute("SELECT * FROM screen where " + where_clause + " order by timestamp asc;")
    recordsInScreenTable = cursor.fetchall()

    # initialize variables
    dateStringForAppUsageList = []
    timestampForAppUsageList = []
    screenTimeValueList = []
    
    print('creating data frame')
    for row in recordsInScreenTable:
        
        ts = row[1]/1000 - 7*60*60 #convert to pacific timezone. ToDo: change fixed value.
        timestampForAppUsageList.append(ts)
        
        datetime_ts = datetime.utcfromtimestamp(ts) 
        dateStringForAppUsageList.append(datetime_ts.strftime('%Y-%m-%d %I:%M:%S %p'))
        screenTimeValueList.append(row[3])
                
    appUsageData = {
        'date': dateStringForAppUsageList,
        'screenTime': screenTimeValueList   
    }
    
    #-- convert to pandas dataframe, with time based indexing
    appUsageDataFrame = pd.DataFrame(appUsageData, columns = ['date','screenTime']) 
    appUsageDataFrame['date'] = pd.to_datetime(appUsageDataFrame['date'], format='%Y-%m-%d %I:%M:%S %p')
    
    return appUsageDataFrame

In [31]:
import mysql.connector as mysql
from datetime import datetime
import time
import json 

today = datetime.today()
s = today.strftime("%m/%d/%Y")
end_unix_time = 1000*time.mktime(datetime.strptime(s + " 02:00 PM", "%m/%d/%Y %I:%M %p").timetuple())
start_unix_time = end_unix_time - 18*3600*1000
user_id = '88315702-a3e6-4296-8437-0a56b4c4f03b'
appUsageDataFrame = getScreenUsageDataFrame(user_id, start_unix_time, end_unix_time)

Fetching data
creating data frame


In [32]:
appUsageDataFrame.head()

,date,screenTime
0,2020-10-31 21:16:45,3
1,2020-10-31 21:16:46,2
2,2020-10-31 21:17:02,3
3,2020-10-31 21:17:37,2
4,2020-10-31 21:41:28,3


In [33]:
appUsageDataFrame.dtypes

date          datetime64[ns]
screenTime             int64
dtype: object

In [41]:
alt.Chart(appUsageDataFrame).mark_circle(size=60).encode(
    x='date:T',
    y={"field": 'screenTime', "type": "quantitative", "scale": {"domain": [1.5,3.5]}}
)

alt.Chart(...)